In [1]:
#surrogate models
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import train_test_split

data_file = '../../../../data_repo/part_aware_data/20000_vectors_drags_sdf.csv'
df = TabularDataset(data_file)

train_df, test_df = train_test_split(df, test_size=0.2, random_state=777)


#exclue the first two columns of train data
train_data = train_df.drop(columns=['i', 'name'])
train_data.head()

/home/xli/anaconda3/envs/surrogate_autogluon/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,...,dim_19992,dim_19993,dim_19994,dim_19995,dim_19996,dim_19997,dim_19998,dim_19999,dim_20000,drag
61,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.375
354,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.374
358,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.435
275,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.437
18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.367


In [2]:
label = 'drag'
print("Summary of class variable: \n", train_data[label].describe())

Summary of class variable: 
 count    351.000000
mean       0.398513
std        0.060013
min        0.278000
25%        0.353000
50%        0.394000
75%        0.435000
max        0.598000
Name: drag, dtype: float64


In [3]:
import os
save_path = './agModels-20000_sdf'  # specifies folder to store trained models
if not os.path.exists(save_path):
    os.makedirs(save_path)

bag_folds = 5 #suggestion range [5, 10]
bag_sets = 3 #suggestion range [1, 20]
stack_levels = 3 #suggestion range [0, 3]
metric = 'root_mean_squared_error' #Regression:mean_absolute_error, mean_squared_error,root_mean_squared_error (default), r2
predictor = TabularPredictor(label=label, path=save_path, eval_metric=metric).fit(train_data, 
                                                                                  presets='best_quality', 
                                                                                  auto_stack="True", 
                                                                                  num_bag_folds=bag_folds, 
                                                                                  num_bag_sets=bag_sets,
                                                                                  num_stack_levels=stack_levels,
                                                                                  verbosity=4)

Presets specified: ['best_quality']
============ fit kwarg info ============
User Specified kwargs:
{'auto_stack': 'True',
 'num_bag_folds': 5,
 'num_bag_sets': 3,
 'num_stack_levels': 3,
 'verbosity': 4}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': None,
 'auto_stack': 'True',
 'calibrate': 'auto',
 'excluded_model_types': None,
 'feature_generator': 'auto',
 'feature_prune_kwargs': None,
 'holdout_frac': None,
 'hyperparameter_tune_kwargs': None,
 'keep_only_best': False,
 'name_suffix': None,
 'num_bag_folds': 5,
 'num_bag_sets': 3,
 'num_stack_levels': 3,
 'pseudo_data': None,
 'refit_full': False,
 'save_space': False,
 'set_best_to_refit_full': False,
 'unlabeled_data': None,
 'use_bag_holdout': False,
 'verbosity': 4}
Stack configuration (auto_stack=True): num_stack_levels=3, num_bag_folds=5, num_bag_sets=3
	Consider setting `time_limit` to ensure training finishes within an expected durat

In [4]:
test_data = test_df.drop(columns=['i', 'name'])
# val_data.head()
y_val = test_data[label]
test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()

,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,...,dim_19991,dim_19992,dim_19993,dim_19994,dim_19995,dim_19996,dim_19997,dim_19998,dim_19999,dim_20000
46,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
175,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
136,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# %%capture log_output
# %config InlineBackend.figure_format = 'retina'
# %config Application.log_level = 'DEBUG'
# %config IPCompleter.greedy = True

predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file
y_pred = predictor.predict(test_data_nolab)
for item in y_pred:
    print(item)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_val, y_pred=y_pred, auxiliary_metrics=True)
print(perf)

results = predictor.fit_summary(show_plot=True)
print(results)
print(predictor.leaderboard(test_data, silent=True))

# with open('./output_5040.log', 'w') as f:
#     f.write(log_output.stdout)

Loading: ./agModels-20000_sdf/predictor.pkl
Loading: ./agModels-20000_sdf/learner.pkl
Loading: ./agModels-20000_sdf/models/trainer.pkl
Loading: ./agModels-20000_sdf/models/CatBoost_BAG_L1/model.pkl
Loading: ./agModels-20000_sdf/models/ExtraTreesMSE_BAG_L1/model.pkl
Loading: ./agModels-20000_sdf/models/LightGBMLarge_BAG_L1/model.pkl
Loading: ./agModels-20000_sdf/models/LightGBMXT_BAG_L1/model.pkl
Loading: ./agModels-20000_sdf/models/LightGBM_BAG_L1/model.pkl
Loading: ./agModels-20000_sdf/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: ./agModels-20000_sdf/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: ./agModels-20000_sdf/models/RandomForestMSE_BAG_L1/model.pkl
Loading: ./agModels-20000_sdf/models/XGBoost_BAG_L1/model.pkl
Loading: ./agModels-20000_sdf/models/CatBoost_BAG_L2/model.pkl
Loading: ./agModels-20000_sdf/models/ExtraTreesMSE_BAG_L2/model.pkl
Loading: ./agModels-20000_sdf/models/LightGBMLarge_BAG_L2/model.pkl
Loading: ./agModels-20000_sdf/models/LightGBMXT_BAG_L2/model.pkl
Loa

0.3858024477958679
0.38262665271759033
0.3744204640388489
0.5213001370429993
0.39365607500076294
0.4229884743690491
0.43686631321907043
0.4890339970588684
0.39392566680908203
0.3345072865486145
0.3631858229637146
0.3926088809967041
0.40933841466903687
0.44797155261039734
0.424248605966568
0.427101194858551
0.4012974500656128
0.3940713405609131
0.3674306869506836
0.35109061002731323
0.43597325682640076
0.42184653878211975
0.37662166357040405
0.38488510251045227
0.47100088000297546
0.4221557676792145
0.4398912191390991
0.3816111087799072
0.41505223512649536
0.37894076108932495
0.4778885841369629
0.3548879623413086
0.3895949721336365
0.4204035997390747
0.40566369891166687
0.3859163522720337
0.3805883526802063
0.43534550070762634
0.4153739809989929
0.39666181802749634
0.4922782778739929
0.3455815017223358
0.49643969535827637
0.3665853440761566
0.4202210605144501
0.39128074049949646
0.4230186641216278
0.38633695244789124
0.3817962408065796
0.3537723422050476
0.42362433671951294
0.3545134663

Loading: ./agModels-20000_sdf/models/NeuralNetFastAI_BAG_L2/model.pkl
Loading: ./agModels-20000_sdf/models/XGBoost_BAG_L2/model.pkl
Loading: ./agModels-20000_sdf/models/NeuralNetTorch_BAG_L2/model.pkl
Loading: ./agModels-20000_sdf/models/LightGBMLarge_BAG_L2/model.pkl
Loading: ./agModels-20000_sdf/models/WeightedEnsemble_L3/model.pkl
Loading: ./agModels-20000_sdf/models/LightGBMXT_BAG_L3/model.pkl
Loading: ./agModels-20000_sdf/models/LightGBM_BAG_L3/model.pkl
Loading: ./agModels-20000_sdf/models/RandomForestMSE_BAG_L3/model.pkl
Loading: ./agModels-20000_sdf/models/CatBoost_BAG_L3/model.pkl
Loading: ./agModels-20000_sdf/models/ExtraTreesMSE_BAG_L3/model.pkl
Loading: ./agModels-20000_sdf/models/NeuralNetFastAI_BAG_L3/model.pkl
Loading: ./agModels-20000_sdf/models/XGBoost_BAG_L3/model.pkl
Loading: ./agModels-20000_sdf/models/NeuralNetTorch_BAG_L3/model.pkl
Loading: ./agModels-20000_sdf/models/LightGBMLarge_BAG_L3/model.pkl
Loading: ./agModels-20000_sdf/models/WeightedEnsemble_L4/model.pkl

*** Summary of fit() ***
Estimated performance of each model:
                     model  score_val  pred_time_val     fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0      WeightedEnsemble_L5  -0.043252     112.053522  8612.280616                0.000883           0.382426            5       True         40
1      WeightedEnsemble_L3  -0.043345      47.847604  6657.513572                0.000747           0.395843            3       True         20
2      WeightedEnsemble_L2  -0.043484       2.953627   143.256142                0.000789           0.462322            2       True         10
3      WeightedEnsemble_L4  -0.043761      81.095035  7479.056296                0.001058           0.407455            4       True         30
4    NeuralNetTorch_BAG_L4  -0.043791     108.576984  8500.466241                1.516268          60.184516            4       True         38
5    NeuralNetTorch_BAG_L2  -0.043858      41.984745  6519.696959         

/home/xli/anaconda3/envs/surrogate_autogluon/lib/python3.10/site-packages/autogluon/core/utils/plots.py:138: UserWarning: AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"
  warnings.warn('AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"')
Loading: ./agModels-20000_sdf/models/LightGBMXT_BAG_L1/model.pkl
Loading: ./agModels-20000_sdf/models/LightGBM_BAG_L1/model.pkl
Loading: ./agModels-20000_sdf/models/RandomForestMSE_BAG_L1/model.pkl
Loading: ./agModels-20000_sdf/models/CatBoost_BAG_L1/model.pkl
Loading: ./agModels-20000_sdf/models/ExtraTreesMSE_BAG_L1/model.pkl
Loading: ./agModels-20000_sdf/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: ./agModels-20000_sdf/models/XGBoost_BAG_L1/model.pkl
Loading: ./agModels-20000_sdf/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: ./agModels-20000_sdf/models/LightGBMLarge_BAG_L1/model.pkl
Loading: 

                     model  score_test  score_val  pred_time_test  \
0   RandomForestMSE_BAG_L1   -0.043148  -0.047416        0.164077   
1     ExtraTreesMSE_BAG_L2   -0.043286  -0.045732       28.693923   
2     ExtraTreesMSE_BAG_L1   -0.043419  -0.047448        0.157441   
3     ExtraTreesMSE_BAG_L3   -0.043439  -0.046285       56.150518   
4          CatBoost_BAG_L1   -0.043466  -0.046246       14.583580   
5          LightGBM_BAG_L2   -0.043528  -0.044448       31.004916   
6           XGBoost_BAG_L1   -0.043607  -0.046880        1.050828   
7   RandomForestMSE_BAG_L2   -0.043678  -0.046202       28.695130   
8          CatBoost_BAG_L2   -0.043774  -0.045300       43.398397   
9     LightGBMLarge_BAG_L2   -0.043810  -0.045274       31.278955   
10          XGBoost_BAG_L2   -0.043820  -0.045373       29.739933   
11       LightGBMXT_BAG_L3   -0.043879  -0.045869       58.289677   
12     WeightedEnsemble_L2   -0.043911  -0.043484        4.783616   
13         LightGBM_BAG_L4   -0.04

In [8]:
best_model = predictor.get_model_best()
print(best_model)

WeightedEnsemble_L5


In [9]:
train_data_pred = predictor.predict(train_data, model=best_model)
test_data_pred = predictor.predict(test_data, model=best_model)

import numpy as np
#save np array y_train_hat to a csv file
np.savetxt('./20000_vectors_y_test_hat_sdf.csv', test_data_pred, delimiter=',')
np.savetxt('./20000_vectors_y_train_hat_sdf.csv', train_data_pred, delimiter=',')

Loading: ./agModels-20000_sdf/models/CatBoost_BAG_L1/model.pkl
Loading: ./agModels-20000_sdf/models/ExtraTreesMSE_BAG_L1/model.pkl
Loading: ./agModels-20000_sdf/models/LightGBMLarge_BAG_L1/model.pkl
Loading: ./agModels-20000_sdf/models/LightGBMXT_BAG_L1/model.pkl
Loading: ./agModels-20000_sdf/models/LightGBM_BAG_L1/model.pkl
Loading: ./agModels-20000_sdf/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: ./agModels-20000_sdf/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: ./agModels-20000_sdf/models/RandomForestMSE_BAG_L1/model.pkl
Loading: ./agModels-20000_sdf/models/XGBoost_BAG_L1/model.pkl
Loading: ./agModels-20000_sdf/models/CatBoost_BAG_L2/model.pkl
Loading: ./agModels-20000_sdf/models/ExtraTreesMSE_BAG_L2/model.pkl
Loading: ./agModels-20000_sdf/models/LightGBMLarge_BAG_L2/model.pkl
Loading: ./agModels-20000_sdf/models/LightGBMXT_BAG_L2/model.pkl
Loading: ./agModels-20000_sdf/models/LightGBM_BAG_L2/model.pkl
Loading: ./agModels-20000_sdf/models/NeuralNetFastAI_BAG_L2/model.pkl
Loadi